In [1]:

import pydra
from pydra.tasks import fsl
from pathlib import Path
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# getting Resolution



patientdir = Path("/Users/rosana.eljurdi/Datasets/Lymphoma")
labeldir = Path(patientdir, "derivatives/manual_segm")


label_paths = list(sorted(labeldir.rglob('*.nii.gz')))
image_paths = list(sorted(set(patientdir.rglob('*.nii.gz')) - set(label_paths)))

ppixdim_info = []
for patient_path in image_paths:
    
    task = fsl.FSLInfo(input_image=patient_path)
    result = task()
   
    ppixdim_info.append({'subject': patient_path.stem, 
                         'ndim1':result.output.pixdim1, 
                         'ndim2': result.output.pixdim2,
                         'ndim3': result.output.pixdim3 })
    
    
df = pd.DataFrame(data=ppixdim_info)
df.to_csv(os.path.join(patientdir,'resolution.csv'))

In [ ]:
#sanity check 1: dropping discarded patients
df_2 = pd.read_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/participants_.tsv", delimiter = '\t')


difff = set(df_2['participant_id'])-set(df['subject'].str.split('_ses-M000').str[0])

df_2 = df_2.set_index('participant_id')
for value in difff:
    df_2.drop(value, inplace=True)
df_2.to_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/participants_.tsv")

In [ ]:
# Sanity Check 2 : Fixing Resolution

df_Res = pd.read_csv("/Users/rosana.eljurdi/Datasets/Lymphoma/resolution.csv")
df_Res['subject'] = df_Res['subject'].str.split('_ses-M000').str[0]
df_Res = df_Res.set_index('subject').drop('Unnamed: 0', axis = 1)

df_Res

In [ ]:

df_2 = pd.read_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/participants_.tsv", delimiter = ',')

df_2 = df_2.set_index('participant_id')

difff = set(df_2.index)-set(df_Res.index)


for value in difff:
    df_2.drop(value, inplace=True)
#df_2.to_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/participants_.tsv")

In [ ]:
# age sex info 

df_AgeSex = pd.read_csv("/Users/rosana.eljurdi/Datasets/Lymphoma/description/age_sex_info.csv")

df_AgeSex['subject'] = df_AgeSex['subject'].apply(lambda x:'sub-{}'.format(x))



df_AgeSex = df_AgeSex.set_index('subject').drop('Unnamed: 0', axis = 1)

df_AgeSex
df_Res

difff = set(df_AgeSex.index)-set(df_Res.index)

difff


for value in difff:
    df_AgeSex.drop(value, inplace=True)
    
df_AgeSex.to_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/age_sex_info.csv")

Meta_data = pd.concat([df_2, df_AgeSex], axis=1)

Meta_data['age'] = Meta_data['age'].str.split('Y').str[0].dropna().astype(int)



In [ ]:
Meta_data.groupby('partition')['partition'].value_counts(normalize=False)

In [ ]:

# age distribution
age_summary = Meta_data.groupby('partition')['age'].mean().to_dict()

age_summary['total'] = Meta_data['age'].mean()

age_summary

In [ ]:
# gender distribution
a = Meta_data.groupby(['partition'])['gender'].value_counts(normalize=False)

a

In [27]:
# magnetic field strength


metadata = pd.read_csv("/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/metadata.csv")
MFS_df = pd.read_csv("/Users/rosana.eljurdi/Datasets/Lymphoma/description/Magnetic_field_strength.csv")

#Meta_data = pd.concat([metadata, MFS_df], axis=1)

metadata.columns = ['subject', 'artefacts', 'lymphome_type','isotropic', 'description', 'color', 'age', 
                   'gender', 'acquisition-date	', 'partition',  'age_list', 'sex_list']

metadata.set_index("subject")

MFS_df.columns = ['to_drop', 'subject','magnetic-field-strength','manufacturer','hospitals' ]

MFS_df.drop('to_drop', axis = 1).set_index("subject")


metadata = pd.concat([metadata, MFS_df], axis=1)
metadata.to_csv('trial')


,subject,artefacts,lymphome_type,isotropic,description,color,age,gender,acquisition-date\t,partition,age_list,sex_list,to_drop,subject,magnetic-field-strength,manufacturer,hospitals
0,sub-1076504,clean,clean,yes,necrotic - hemorragy ! hystology? EBV ?,yellow,32.0,M,2016-12-16,D1,032Y,M,0,sub-1078624,3.0,GE,CH Pitie Salpetriere Pr CHIRAS
1,sub-1071513,clean,clean,yes,peripheral - nice,yellow,77.0,F,2011-10-24,D1,077Y,F,1,sub-1071101,1.5,GE,CH_PITIE_SALPETRIERE
2,sub-1075729,clean,clean,yes,very low contrast enhancement,orange,55.0,M,2014-06-06,D3,055Y,M,2,sub-1073847,1.5,GE,CH_PITIE_SALPETRIERE
3,sub-1072353,NaN,NaN,NaN,conversion to nii problem,red,72.0,F,2020-06-30,D4,072Y,F,3,sub-1070570,1.5,GE,INSTITUT_CURIE
4,sub-1073888,clean,clean,yes,peripheral and infratentorial,yellow,83.0,F,2014-02-07,D1,083Y,F,4,sub-1071364,3.0,GE,CH Pitie Salpetriere Pr CHIRAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,sub-1078418,clean,clean,yes,typical - peripheric,yellow,65.0,M,2020-03-16,D1,065Y,M,113,sub-1071219,3.0,GE,CH Pitie Salpetriere Pr CHIRAS
114,sub-1071798,clean,treated,no,peripheral,yellow,86.0,F,2012-08-01,D1,086Y,F,114,sub-1071155,3.0,GE,NEURO RADIO GHPS
115,sub-1078847,clean,clean,yes,typical - peripheric,yellow,62.0,F,2019-11-22,D1,062Y,F,115,sub-1071384,3.0,GE,NEURO RADIO GHPS
116,sub-1073458,clean,treated,yes,peripheral,yellow,68.0,F,2018-09-10,D1,068Y,F,116,sub-1071135,3.0,Siemens,IRM3
